In [1]:
# load npy
import numpy as np
import matplotlib.pyplot as plt

model = np.load('model.npy')
data = np.load('data.npy')


In [31]:
import pypose as pp
def plot_trajctory(pose, arr_length=0.8, trajectory_length=3,title = '',feq = 2,plot_3d_pose = True,limit = 10.0,
                   create_new_figure=True,fig=None):
    # Plot the 3D pose
    if create_new_figure:
        fig = plt.figure(figsize=(8, 8))
    ax = fig.add_subplot(111, projection='3d')
        # Define the 3D pose (translation and rotation)
    # arr_length = 0.8
    for i in range(0,trajectory_length,feq):
        if plot_3d_pose:
            R = pp.SO3(pose[i,3:]).matrix().numpy()
            translation = pose[i,:3]

            x_axis = np.dot(R, np.array([1, 0, 0]))
            y_axis = np.dot(R, np.array([0, 1, 0]))
            z_axis = np.dot(R, np.array([0, 0, 1]))


            ax.quiver(*translation, *x_axis, color='r', label='X-axis' , length=arr_length)
            ax.quiver(*translation, *y_axis, color='g', label='Y-axis' , length=arr_length)
            ax.quiver(*translation, *z_axis, color='b', label='Z-axis' , length=arr_length)
            # ax.quiver(*(np.array([0, 0, 0])), *translation, color='k', label='Origin' , length=1)
            ax.text(*translation, str(i)
                    , color='k')

        else:
            try:
                pose = pose.numpy()
            except:
                pass
            ax.plot(pose[:trajectory_length,0], pose[:trajectory_length,1], pose[:trajectory_length,2], 
                    # color='k', 
                    label='Trajectory')
    
    ax.set_xlim([-limit, limit])
    ax.set_ylim([-limit, limit])
    ax.set_zlim([-limit, limit])
    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    ax.set_zlabel('Z')
    # title
    ax.set_title(title)

In [10]:
model - model.mean(1).reshape(3,1)

array([[-3.864, -3.878, -3.892, ..., -3.793, -3.787, -3.779],
       [-3.299, -3.298, -3.298, ..., -3.456, -3.467, -3.476],
       [ 1.111,  1.074,  1.037, ...,  1.231,  1.237,  1.244]])

In [5]:
"""Align two trajectories using the method of Horn (closed-form).

Input:
model -- first trajectory (3xn)
data -- second trajectory (3xn)

Output:
rot -- rotation matrix (3x3)
trans -- translation vector (3x1)
trans_error -- translational error per point (1xn)

"""
calc_scale = True
import numpy
numpy.set_printoptions(precision=3,suppress=True)
model_zerocentered = model - model.mean(1).reshape(3,1)
data_zerocentered = data - data.mean(1).reshape(3,1)

W = numpy.zeros( (3,3) )
for column in range(model.shape[1]):
    W += numpy.outer(model_zerocentered[:,column],data_zerocentered[:,column])
U,d,Vh = numpy.linalg.linalg.svd(W.transpose())
S = numpy.matrix(numpy.identity( 3 ))
if(numpy.linalg.det(U) * numpy.linalg.det(Vh)<0):
    S[2,2] = -1
rot = U*S*Vh

if calc_scale:
    rotmodel = rot*model_zerocentered
    dots = 0.0
    norms = 0.0
    for column in range(data_zerocentered.shape[1]):
        dots += numpy.dot(data_zerocentered[:,column].transpose(),rotmodel[:,column])

        # dot = numpy.linalg.norm(rotmodel[:,column])
        # dots += dot * dot
        
        normi = numpy.linalg.norm(model_zerocentered[:,column])
        norms += normi*normi
    # s = float(dots/norms)  
    s = float(norms/dots)
else:
    s = 1.0  
print(s)

3.515964319933332


In [7]:
model_zerocentered.shape

(3, 3637)

In [15]:

import matplotlib
matplotlib.use('WebAgg')



In [22]:

rotmodel = np.array(rotmodel)


In [32]:
plt.close('all')
trajectory_length = model_zerocentered.shape[1]
# fig = plt.figure(figsize=(8, 8))
fig = None
plot_trajctory(model_zerocentered.transpose(),trajectory_length=trajectory_length,plot_3d_pose=False,limit=10.0,
               title='model_zerocentered',fig = fig)
plot_trajctory(data_zerocentered.transpose(),trajectory_length=trajectory_length,plot_3d_pose=False,limit=10.0,
               title='data_zerocentered',fig = fig)
plot_trajctory(rotmodel.transpose(),trajectory_length=trajectory_length,plot_3d_pose=False,limit=10.0,
               title='s*rot*model_zerocentered',fig = fig)
plt.show()


In [ ]:
data_root + str(0) + "/data/1403638127345096960.png"

'/home/data2/euroc_raw/MH_04_difficult/mav0/cam0/data/1403638127345096960.png'